In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 
import missingno as msno
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from lightgbm import LGBMRegressor
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data =  pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/data.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv",index_col="row-col")

In [ ]:
data.shape

In [ ]:
data.head()

**EDA**

In [ ]:
data.describe()

In [ ]:
msno.matrix(data.sample(500))

In [ ]:
corr = data.corr().abs()
plt.figure(figsize=(10,8))
sns.heatmap(corr,vmax=1,vmin=-1,center=0)

**Model**

In [ ]:
imp = IterativeImputer(
    estimator=LGBMRegressor(
    n_estimators=100,
    random_state=42,
    ),
    missing_values=np.nan,
    max_iter=10,
    initial_strategy='mean',
    imputation_order='ascending',
    verbose=2,
    random_state=42
)

data[:] = imp.fit_transform(data)

**Submission**

In [ ]:
for i in tqdm(sub.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    sub.loc[i, 'value'] = data.loc[row, col]
    
sub.to_csv('submission.csv')